# TLE - Brute force - Iterative

Compute all subarrays and their product, retain max product.  
Time complexity: $O(n^3)$ when the entire product of `nums[i:j]` is recomputed for every $0 \leq i < j \leq len(nums)$. If a running product is used, then the time complexity drops to $O(n²)$.

In [7]:
from functools import reduce

def brute_force_cube(nums: list[int]) -> int:
    res = float("-inf")
    for i in range(len(nums)):
        for j in range(i + 1, len(nums) + 1):
            subproduct = reduce(lambda x, y: x * y, nums[i:j])
            res = max(res, subproduct)
    return res

def brute_force_square(nums: list[int]) -> int:
    res = float("-inf")
    for i in range(len(nums)):
        subproduct = 1
        for j in range(i, len(nums)):
            subproduct *= nums[j] # Lowers the time complexity from O(n3) to O(n2)
            res = max(res, subproduct)
    return res

nums1 = [2,3,-2,4]
nums2 = [-2,0,-1]
brute_force_cube(nums1), brute_force_cube(nums2)

(6, 0)

In [8]:
brute_force_square(nums1), brute_force_square(nums2)

(6, 0)

# TLE - Brute force - Recursive

Both iterative brute force solutions compute 'overlapping' products (factor-wise) in each outer loop, e.g. for $a = [a_0, a_1, a_2]$:  

$a_0, \, a_0 \cdot a_1, \, a_0 \cdot a_1 \cdot a_2$  
$\qquad \quad \, a_1, \quad \ \ \, \, a_1 \cdot a_2$  
$\qquad \qquad \qquad \qquad a_2$

The above layout of the products suggests there might be a way to leverage information from indices $j > i$ in order to compute a result for index $i$, i.e recursively from one row to the row above. For instance, if we have determined that $a_1 \cdot a_2 > 0$ , then we don't need to compute the first 3 products:
* If $a_0 > 0$, then the result is $a_0 \cdot (a_1 \cdot a_2)$
* If $a_0 = 0$, then the result is $a_1 \cdot a_2$
* If $a_0 < 0$, then the result is $a_1 \cdot a_2$

This operation is $O(1)$, as opposed to $O(n)$ for the computation of all the products of the subarrays starting with $a_0$.  

Unfortunately, we can not tackle the exact problem at hand recursively. Take `nums = [2, -1, 3]` and say `f` is a recursive function that returns something like the maximum subarray product, or a valid subarray, or both. `f([-1, 3])` is 3 with subarray `[3]`. But that does not help to recursively determine `f([2, -1, 3])`! Because `2 * f([-1, 3])` = `2 * 3` = `6` is not a valid answer: the `2` and the `3` are not contiguous! We're stuck.  

Wait, what if `f(i)` returned the maximum product for subarrays **starting at index i**? Ok, but we want the maximum product for all subarrays, not just the ones starting at index `i`. Well, if we compute `f(i)` for each starting index `i`, then the maximum product subarray is $max \{ f(i) / 0 \leq i < N\}$.

I won't detail the inner behavior of `f` which is detailed in a lot of solutions including the editorial.

In [9]:
def f(i: int) -> tuple[int, int]:
    if i == len(nums) - 1:
        return nums[-1], nums[-1]
    m, M = f(i + 1)
    curr = nums[i]
    new_m = min(curr, curr * m, curr * M)
    new_M = max(curr, curr * m, curr * M)
    return new_m, new_M

def mps_rec_squared(nums):
    return max([f(i)[1] for i in range(len(nums))])

nums = [2, 3, -2, 4]
mps_rec_squared(nums)

6

# Recursive

To lower the time complexity down to $O(n²)$, take advantage of the recursive calls to compute the current max up until 'now' (meaning now in the call stack).

In [10]:
def f(i: int) -> tuple[int, int, int]:
    # Base case
    if i == len(nums) - 1:
        last = nums[-1]
        return last, last, last
    
    # Recursion
    m, M, max_so_far = f(i + 1)
    curr = nums[i]
    new_m = min(curr, curr * m, curr * M)
    new_M = max(curr, curr * m, curr * M)
    max_so_far = max(max_so_far, new_M)
    return new_m, new_M, max_so_far

def recursive_mps(nums: list[int]) -> int:
    return f(0)[-1]

nums = [2,3,-2,4]
recursive_mps(nums)

6

# Dynamic Programming

Start from the base case and iteratively climb up the call stack. The order of the processing does not have any consequence on the result so we can reverse the logic and go from left to right.

def 